### Preparación de datos

In [1]:
from pathlib import Path
import pandas as pd

# === RUTAS ===

ruta_acta =  Path(r"20251003/20251003/ADP_DTM_FACT.Acta.csv")
ruta_insumos =  Path(r"20251003/20251003/ADP_DTM_DIM.Insumo.csv")
ruta_proyectos =  Path(r"20251003/20251003/ADP_DTM_DIM.Proyecto.csv")
ruta_capitulos = Path(r"20251003/20251003/ADP_DTM_DIM.CapituloPresupuesto.csv")
ruta_proyeccion =  Path(r"20251003/20251003/ADP_DTM_FACT.Proyeccion.csv")
ruta_items =  Path(r"20251003/20251003/ADP_DTM_DIM.Items.csv")




In [2]:
# === CARGA ===
tabla_proyeccion = pd.read_csv(ruta_proyeccion)
tabla_items = pd.read_csv(ruta_items)
tabla_proyectos = pd.read_csv(ruta_proyectos)
tabla_capitulos = pd.read_csv(ruta_capitulos)
tabla_insumos = pd.read_csv(ruta_insumos)

C:\Users\aleja\AppData\Local\Temp\ipykernel_6912\2920138097.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  tabla_items = pd.read_csv(ruta_items)


### Un join

In [7]:
tabla_base = tabla_proyeccion.copy()

# Merges previos
tabla_1 = pd.merge(tabla_base, tabla_proyectos, on="SkIdProyecto", how="left")
tabla_2 = pd.merge(tabla_1, tabla_capitulos, on="SkIdCapitulo", how="left")

# Asegura unicidad en Items
tabla_items_unica = tabla_items.drop_duplicates(subset=["SkIdItems"], keep="first")
tabla_3 = pd.merge(
    tabla_2, tabla_items_unica,
    on="SkIdItems", how="left",
    suffixes=("", "_item")   # control de sufijos para items
)

# 🔧 FIX insumos: prefijar y garantizar clave única
tabla_insumos_unica = tabla_insumos.drop_duplicates(subset=["SkIdInsumo"], keep="first").copy()
cols_no_clave_insumo = [c for c in tabla_insumos_unica.columns if c != "SkIdInsumo"]
tabla_insumos_pref = tabla_insumos_unica.rename(
    columns={c: f"Insumo_{c}" for c in cols_no_clave_insumo}
)

# (Opcional) Debug rápido de posibles overlaps antes del merge final
overlap_prev = set(tabla_3.columns).intersection(set(tabla_insumos_pref.columns))
print(f"[DEBUG] Overlap antes de merge con insumos (debería ser solo 'SkIdInsumo'): {overlap_prev}")

# Merge final (sin sufijos porque ya prefijamos)
tabla_4 = pd.merge(
    tabla_3, tabla_insumos_pref,
    on="SkIdInsumo", how="left",
    validate="many_to_one"
)

print(f"[DEBUG] OK. Forma final: {tabla_4.shape}")

tabla_4.to_csv(
    "tabla_looker.csv",
    index=False,
    encoding="utf-8",
    sep=","                
)

[DEBUG] Overlap antes de merge con insumos (debería ser solo 'SkIdInsumo'): {'SkIdInsumo'}
[DEBUG] OK. Forma final: (273450, 107)


KeyboardInterrupt: 

In [8]:
# === SELECCIÓN DE COLUMNAS ===
columnas_finales = [
    "SkIdProyecto", "SkIdCapitulo", "SkIdItems", "SkIdInsumo",
    "Nombre Proyecto", "Capitulo Descripcion", "Item Descripcion", "Insumo_Insumo Descripcion", "Insumo_Agrupacion Descripcion"
    "SkIdFecha Real", "Cantidad", "Valor Unitario", "Valor Total", "Insumo_Valor Unitario", "Insumo_Valor Neto", "Insumo_Fecha Creacion",
    "Cantidad Item", "Macroproyecto Descripcion", "Insumo_Fecha Modificacion",
    "Fecha De Elaboracion", "Fecha De Inicio", "Fecha De Finalización", "SkIdFecha", "Capitulo Numero", "Cantidad_Item"
]

# Filtrar solo las columnas que existan realmente (por seguridad)
columnas_existentes = [col for col in columnas_finales if col in tabla_4.columns]
tabla_looker = tabla_4[columnas_existentes].copy()

#### Debug

# Parte dos tabla final 

In [1]:
import pandas as pd
from datetime import datetime
import setUp

# === RUTAS ===
ruta_tabla_proyeccion = setUp.path_proyeccion
ruta_tabla_items = setUp.path_items
ruta_tabla_proyectos = setUp.path_proyectos
ruta_tabla_capitulos = setUp.path_capitulos
ruta_tabla_insumos = setUp.path_insumos
ruta_tabla_controlProyecto = setUp.path_controlProyecto

# === RUTA DE SALIDA ===
ruta_salida = setUp.path_salida


## Importar tablas necesarias

In [2]:
proyeccion = pd.read_csv(ruta_tabla_proyeccion, low_memory=False)
controlProyecto = pd.read_csv(ruta_tabla_controlProyecto, low_memory=False)
proyecto = pd.read_csv(ruta_tabla_proyectos, low_memory=False)
capitulos = pd.read_csv(ruta_tabla_capitulos, low_memory=False)
items = pd.read_csv(ruta_tabla_items, low_memory=False)
insumos = pd.read_csv(ruta_tabla_insumos, low_memory=False)

In [3]:
display(proyeccion.shape,
        controlProyecto.shape)

(273450, 19)

(1620134, 13)

In [4]:
controlProyecto.columns

Index(['SkIdEmpresa', 'Empresa', 'SkIdProyecto', 'SkIdFecha',
       'SkIdClaseOrigen', 'SkIdInsumo', 'SkIdCapitulo', 'SkIdItems',
       'Cantidad', 'Valor Total', 'Origen Documento',
       'Origen Documento Detalle', 'Valor Sin IVA'],
      dtype='object')

In [5]:
proyeccion.columns

Index(['SkIdEmpresa', 'SkIdProyecto', 'SkIdCapitulo', 'SkIdItems',
       'SkIdInsumo', 'SkIdReforma', 'SkIdUsuario', 'SkIdFecha',
       'SkIdFecha Real', 'SkIdEstado', 'Cantidad', 'Valor Unitario',
       'Valor Total', 'Origen', 'Causa', 'Cantidad Item', 'Descripcion Causa',
       'Ajuste Global', 'Empresa'],
      dtype='object')

### Revisión

Se elimina de ControlProjecto los `SkIdProyecto` que no están en Proyección esto con el motivo de poder cruzar las tablas posteriormente sin perder información, el resultado son `76` proyectos, habrá que revisar si estos proyectos son los mismos `76` que inicialmente `ARPRO` brindo. Al momento de realizar el filtro se quedó con `1607913` registros de los `1620134` Continuamos a agregar la descripción del proyecto y otras columnas en `Control_filtrado`.

In [6]:
# Filtrar ControlProjecto para conservar solo los SKIProjecto presentes en Proyección
Control_filtrado = controlProyecto[
    controlProyecto["SkIdProyecto"].isin(proyeccion["SkIdProyecto"])
]


In [ ]:
Control_filtrado.shape
1607913 - 1620134
# len(Control_filtrado['SkIdProyecto'].unique())

(1607913, 13)

## Proyecto

In [7]:
proyecto.columns

Index(['SkIdProyecto', 'Codigo Proyecto', 'Nombre Proyecto', 'Clase Proyecto',
       'Tipo', 'Estado', 'Presupuesto Fijo', 'Propietario', 'Sucursal',
       'Sucursal Nombre', 'MacroProyecto', 'MacroProyecto Descripcion',
       'Centro Costo', 'Centro Costo Descripcion', 'VIS',
       'Sucursal Administrativa', 'SkIdEmpresa', 'Empresa',
       'Fecha De Elaboracion', 'Fecha De Inicio', 'Fecha De Finalizacion',
       'UnidadAConstruir', 'CantidadAConstruir', 'AreaAConstruir_M2',
       'AreaConstruidaFinal_M2', 'AreaAVender_M2', 'Arealote_M2',
       'CostoPreFactibilidad', 'Iniciales', 'Nocontrato', 'Cliente',
       'Inversionista', 'Almacenista', 'PorcentajeAdministracion',
       'PorcentajeImprevistos', 'PorcentajeUtilidad', 'IVA'],
      dtype='object')

### Merge

In [8]:
# Control_NP := Control nombre proyecto
columnas_proyecto = [
    "SkIdProyecto",
    "Nombre Proyecto",
    "Clase Proyecto",
    "MacroProyecto Descripcion",
    "VIS",
    "Fecha De Elaboracion",
    "Fecha De Inicio",
    "Fecha De Finalizacion",
    "UnidadAConstruir",
    'CantidadAConstruir',
    'AreaAConstruir_M2',
    'AreaConstruidaFinal_M2',
    'AreaAVender_M2',
    'Arealote_M2',
    "PorcentajeImprevistos",
    "PorcentajeUtilidad",
    "IVA"
]
Control_NP = Control_filtrado.merge(
    proyecto[columnas_proyecto],  # columnas que quieres traer
    on="SkIdProyecto",
    how="left"  # conserva todos los registros de Control_filtrado
)


In [9]:
Control_NP.shape

(1607913, 29)

In [46]:
Control_NP.columns

Index(['SkIdEmpresa', 'Empresa', 'SkIdProyecto', 'SkIdFecha',
       'SkIdClaseOrigen', 'SkIdInsumo', 'SkIdCapitulo', 'SkIdItems',
       'Cantidad', 'Valor Total', 'Origen Documento',
       'Origen Documento Detalle', 'Valor Sin IVA', 'Nombre Proyecto',
       'MacroProyecto Descripcion', 'VIS', 'Fecha De Elaboracion',
       'Fecha De Inicio', 'Fecha De Finalizacion', 'UnidadAConstruir',
       'CantidadAConstruir', 'AreaAConstruir_M2', 'AreaConstruidaFinal_M2',
       'AreaAVender_M2', 'Arealote_M2', 'PorcentajeImprevistos',
       'PorcentajeUtilidad', 'IVA'],
      dtype='object')

In [35]:
# Reordenar columnas para mejor visualización

## Guardar la columna temporalmente
col = Control_NP.pop("Nombre Proyecto")

## Insertarla después de 'SkIdProyecto' (índice basado en posición)
pos = Control_NP.columns.get_loc("SkIdItems") + 1
Control_NP.insert(pos, "Nombre Proyecto", col)

Ahora procedemos a agregar la `descripción` y `número` de los capítulos.

## Capitulos

In [12]:
controlProyecto.columns

Index(['SkIdEmpresa', 'Empresa', 'SkIdProyecto', 'SkIdFecha',
       'SkIdClaseOrigen', 'SkIdInsumo', 'SkIdCapitulo', 'SkIdItems',
       'Cantidad', 'Valor Total', 'Origen Documento',
       'Origen Documento Detalle', 'Valor Sin IVA'],
      dtype='object')

In [10]:
capitulos.columns

Index(['SkIdCapitulo', 'SkIdEmpresa', 'Codigo Proyecto', 'Capitulo Numero',
       'Capitulo Descripcion', 'Tipo Costo', 'Tipo Costo Orden', 'Empresa'],
      dtype='object')

### Merge

In [13]:
# Control_NNC := Nombre número capítulo
columnas_capitulo = [
    "SkIdCapitulo",
    "Capitulo Numero",
    "Capitulo Descripcion",
    "Tipo Costo"
]
Control_NNC = Control_NP.merge(
    capitulos[columnas_capitulo],  # columnas que quieres traer
    on="SkIdCapitulo",
    how="left"  # conserva todos los registros de Control_filtrado
)

In [14]:
Control_NNC.shape

(1607913, 32)

## Items

### Revisión

Hay Items únicos en la tabla `controlProyecto` que no estan en la tabla `Items`e items únicos que no estan la tabla `proyeccion` de la tabla `controlProyecto`. Además, tambien hay items únicos en la tabla `items` que no estan 

In [22]:
display(len(set(Control_filtrado['SkIdItems']).difference(set(items['SkIdItems']))),
len(set(Control_filtrado['SkIdItems']).difference(set(proyeccion['SkIdItems']))),
len(set(items['SkIdItems']).difference(set(Control_filtrado['SkIdItems']))),
len(set(items['SkIdItems']).difference(set(proyeccion['SkIdItems']))))

13486

25687

4572

16773

In [36]:
len(set(zip(items['SkIdAPU'],items['SubCapitulo'], items['Item Descripcion'])))

49402

Si juntamos a `SkIdAPU`, `SubCapitulo` e `Item Descripcion` obtenemos **49,402** registros diferentes. Esto se calculo en consecuencia de que en esta tabla (`Items`) tenemos **49,311** registros, sin embargo, tenemos **23,786** descripciones únicas, por lo que se le intentó buscar una justificación a esta dimensión.

In [37]:
items.columns

Index(['SkIdItems', 'SkIdEmpresa', 'SkIdAPU', 'SkIdNivel', 'Empresa',
       'Item No', 'SubCapitulo', 'Item Descripcion', 'Cantidad',
       'Valor Sin IVA', 'Precio Venta', 'Codigo Cliente',
       'Cantidad Proyectada', 'Unidad Medida', 'Item estado', 'Metro cuadrado',
       'Aplica En Contratos', 'Aplica En Almacen',
       'Bloqueo De Contratos Por Cantidad', 'Bloqueo De Contratos Por Valor',
       'Bloqueo De Salidas Por Cantidad', 'Bloqueo De Salidas Por Valor',
       'Clase Item'],
      dtype='object')

### Merge

In [40]:
columnas_items = [
    "SkIdItems",
    "SubCapitulo",
    "Item Descripcion",
    "Cantidad",
    "Valor Sin IVA",
    "Precio Venta",
    'Codigo Cliente',
    'Cantidad Proyectada',
    'Unidad Medida',
    'Item estado',
    'Bloqueo De Contratos Por Valor',
    'Bloqueo De Salidas Por Cantidad',
    'Bloqueo De Salidas Por Valor',
]
# Control_I := Control items
Control_I = Control_NNC.merge(
    items[columnas_items],  # columnas que quieres traer
    on="SkIdItems",
    how="left"  # conserva todos los registros de Control_filtrado
)
Control_I.shape

(1607913, 44)

In [ ]:
# Cambio nombre columna cantidad de item y control.
columnas_items = [
    "SkIdItems",
    "SubCapitulo",
    "Item Descripcion",
    "Cantidad",
    "Valor Sin IVA",
    "Precio Venta",
    'Codigo Cliente',
    'Cantidad Proyectada',
    'Unidad Medida',
    'Item estado',
    'Bloqueo De Contratos Por Valor',
    'Bloqueo De Salidas Por Cantidad',
    'Bloqueo De Salidas Por Valor',
]



Control_I = Control_I.rename(columns={"Cantidad_y": "Cantidad_Item", 
                                      "Cantidad_x": "Cantidad_Control",
                                      "Valor Sin IVA": "ValorSinIVA_Item",
                                      "Precio Venta": "PrecioVenta_Item",
                                      "Unidad Medida": "UnidadMedida_Item",
                                      "Cantidad Proyectada": "CantidadProyectada_Item"
                                     })

In [43]:
Control_I.columns

Index(['SkIdEmpresa', 'Empresa', 'SkIdProyecto', 'SkIdFecha',
       'SkIdClaseOrigen', 'SkIdInsumo', 'SkIdCapitulo', 'SkIdItems',
       'Cantidad_Control', 'Valor Total', 'Origen Documento',
       'Origen Documento Detalle', 'Valor Sin IVA_x', 'Nombre Proyecto',
       'Clase Proyecto', 'MacroProyecto Descripcion', 'VIS',
       'Fecha De Elaboracion', 'Fecha De Inicio', 'Fecha De Finalizacion',
       'UnidadAConstruir', 'CantidadAConstruir', 'AreaAConstruir_M2',
       'AreaConstruidaFinal_M2', 'AreaAVender_M2', 'Arealote_M2',
       'PorcentajeImprevistos', 'PorcentajeUtilidad', 'IVA', 'Capitulo Numero',
       'Capitulo Descripcion', 'Tipo Costo', 'SubCapitulo', 'Item Descripcion',
       'Cantidad_Item', 'Valor Sin IVA_y', 'Precio Venta', 'Codigo Cliente',
       'Cantidad Proyectada', 'Unidad Medida', 'Item estado',
       'Bloqueo De Contratos Por Valor', 'Bloqueo De Salidas Por Cantidad',
       'Bloqueo De Salidas Por Valor'],
      dtype='object')

## Insumos

In [45]:
insumos.columns

Index(['SkIdInsumo', 'SkIdEmpresa', 'Empresa', 'Codigo Insumo',
       'Insumo Descripcion', 'Agrupacion', 'Agrupacion Descripcion', 'Tipo',
       'Tipo Descripcion', 'Unidad', 'Descripcion Unidad', 'Estado',
       'Requiere Equipo', 'Dias Reposicion', 'SubAnalisis', 'Devolutivo',
       'Stock Maximo', 'Stock Minimo', 'Valor Unitario', 'Porcentaje IVA',
       'Valor Neto', 'Fecha Creacion', 'Fecha Modificacion',
       'Codigo Insumo Id'],
      dtype='object')

### Merge

In [52]:
columnas_insumos = [
    "SkIdInsumo",
    "Insumo Descripcion",
    'Agrupacion Descripcion',
    'Tipo Descripcion',
    'Descripcion Unidad',
    'Valor Unitario',
    'Porcentaje IVA',
    'Valor Neto',
    'Fecha Creacion',
    'Fecha Modificacion'
]
# Control_In := Control de insumos
Control_In = Control_I.merge(
    insumos[columnas_insumos],  # columnas que quieres traer
    on="SkIdInsumo",
    how="left"  # conserva todos los registros de Control_filtrado
)
Control_In.shape

(1607913, 53)

In [53]:
# Renombrar columnas
Control_In = Control_In.rename(columns={"Agrupacion Descripcion": "Agrupacion_Desc_Insumo", 
                                      "Tipo Descripcion": "TipoDescripcion_Insumo",
                                      "Descripcion Unidad": "DescripcionUnidad_Insumo",
                                      "Valor Unitario": "ValorUnitario_Insumo",
                                      "Porcentaje IVA": "PorcentajeIVA_Insumo",
                                      "Valor Neto": "ValorNeto_Insumo",
                                      "Fecha Creacion": "FechaCreacion_Insumo",
                                      "Fecha Modificacion": "FechaModificacion_Insumo"
                                     })

In [54]:
Control_In.columns

Index(['SkIdEmpresa', 'Empresa', 'SkIdProyecto', 'SkIdFecha',
       'SkIdClaseOrigen', 'SkIdInsumo', 'SkIdCapitulo', 'SkIdItems',
       'Cantidad_Control', 'Valor Total', 'Origen Documento',
       'Origen Documento Detalle', 'Valor Sin IVA_x', 'Nombre Proyecto',
       'Clase Proyecto', 'MacroProyecto Descripcion', 'VIS',
       'Fecha De Elaboracion', 'Fecha De Inicio', 'Fecha De Finalizacion',
       'UnidadAConstruir', 'CantidadAConstruir', 'AreaAConstruir_M2',
       'AreaConstruidaFinal_M2', 'AreaAVender_M2', 'Arealote_M2',
       'PorcentajeImprevistos', 'PorcentajeUtilidad', 'IVA', 'Capitulo Numero',
       'Capitulo Descripcion', 'Tipo Costo', 'SubCapitulo', 'Item Descripcion',
       'Cantidad_Item', 'Valor Sin IVA_y', 'PrecioVenta_Item',
       'Codigo Cliente', 'CantidadProyectada_Item', 'UnidadMedida_Item',
       'Item estado', 'Bloqueo De Contratos Por Valor',
       'Bloqueo De Salidas Por Cantidad', 'Bloqueo De Salidas Por Valor',
       'Insumo Descripcion', 'Agrupa

# Obsevaciones

1. Se encontró que en ciertos proyectos no había ningún pedido registrado. Las tablas relacionadas a esta observación `especificacionpedidos` y `proyecto`.
2. 


# Tareas por hacer

La tabla base es la última que obtuvo cambios con algún merge, en esta versión `Control_In`

- Revisar cual de las tablas `controlProyecto` o `proyeccion` se relaciona más con el flujo que realizó Rafa
- Revisar como esta tabla se puede relacionar con la tabla `Actas`
- Falta generar el merge con `ClaseOrigen`
- Falta generar el merge con `ejecucion`.
- Merge `controlProyecto` con `proyección` y así revisar la proporsión de la intersección respecto a la unón. (interseccion/union -> Registros)
- Con respecto a la anterior tarea revisar la siguiente idea:

>Se piensa que hay puede haber un sistema de variables asociadas a `controlProyecto` que no coincida total o parcialmente con un sistema de variables en la tabla `proyección`
 
- En función a la observación `1`, revisar si los proyectos que estan en pedidos estan más asociados con las tablas `controlProyecto` o `proyeccion`.


In [37]:
import pandas as pd
from datetime import datetime
import setUp

# === RUTAS ===
ruta_tabla_proyeccion = setUp.path_proyeccion
ruta_tabla_items = setUp.path_items
ruta_tabla_proyectos = setUp.path_proyectos
ruta_tabla_capitulos = setUp.path_capitulos
ruta_tabla_insumos = setUp.path_insumos
ruta_tabla_controlProyecto = setUp.path_controlProyecto

# === RUTA DE SALIDA ===
ruta_salida = setUp.path_salida

# === 1) PROYECCION ===
proyeccion = pd.read_csv(ruta_proyeccion, low_memory=False)
columnas_proyeccion = [
    "SkIdProyecto", "SkIdCapitulo", "SkIdItems", "SkIdInsumo",
    "SkIdFecha", "SkIdFecha Real", "Cantidad", "Valor Unitario", "Valor Total"
]
missing = [c for c in columnas_proyeccion if c not in proyeccion.columns]
if missing:
    raise KeyError(f"Faltan columnas en Proyeccion: {missing}")

proyeccion_filtrada = proyeccion[columnas_proyeccion].rename(columns={
    "Cantidad": "Cantidad_Insumo",
    "Valor Unitario": "Valor_Unitario_Insumo",
    "Valor Total": "Valor_Total_Item"
})

# === 2) PROYECTO ===
proyecto = pd.read_csv(ruta_proyectos, low_memory=False)
columnas_proyecto = [
    "SkIdProyecto", "Codigo Proyecto", "Nombre Proyecto", "Clase Proyecto", "Estado",
    "MacroProyecto Descripcion", "Fecha De Elaboracion", "Fecha De Inicio", "Fecha De Finalizacion"
]
missing = [c for c in columnas_proyecto if c not in proyecto.columns]
if missing:
    raise KeyError(f"Faltan columnas en Proyecto: {missing}")

proyecto_filtrado = proyecto[columnas_proyecto].rename(columns={
    "Codigo Proyecto": "Codigo_Proyecto",
    "Nombre Proyecto": "Nombre_Proyecto",
    "Clase Proyecto": "Clase_Proyecto",
    "Estado": "Estado_Proyecto",
    "MacroProyecto Descripcion": "MacroProyecto_Descripcion_Proyecto",
    "Fecha De Elaboracion": "Fecha_De_Elaboracion_Proyecto",
    "Fecha De Inicio": "Fecha_De_Inicio_Proyecto",
    "Fecha De Finalizacion": "Fecha_De_Finalizacion_Proyecto"
})

merge_proyeccion_proyecto = pd.merge(
    proyeccion_filtrada, proyecto_filtrado,
    on="SkIdProyecto", how="left", validate="m:1"
)

# === 3) ITEMS ===
items = pd.read_csv(ruta_items, low_memory=False)
columnas_items = ["SkIdItems", "Item Descripcion"]
missing = [c for c in columnas_items if c not in items.columns]
if missing:
    raise KeyError(f"Faltan columnas en Items: {missing}")

items_filtrado = items[columnas_items].rename(columns={"Item Descripcion": "Item_Descripcion"})

merge_items = pd.merge(
    merge_proyeccion_proyecto, items_filtrado,
    on="SkIdItems", how="left", validate="m:1"
)

# === 4) INSUMO ===
insumo = pd.read_csv(ruta_insumos, low_memory=False)
columnas_insumo = ["SkIdInsumo", "Insumo Descripcion", "Fecha Creacion", "Fecha Modificacion"]
missing = [c for c in columnas_insumo if c not in insumo.columns]
if missing:
    raise KeyError(f"Faltan columnas en Insumo: {missing}")

insumo_filtrado = insumo[columnas_insumo].rename(columns={
    "Insumo Descripcion": "Insumo_Descripcion",
    "Fecha Creacion": "Fecha_Creacion_Insumo",
    "Fecha Modificacion": "Fecha_Modificacion_Insumo"
})

tabla_intermedia = pd.merge(
    merge_items, insumo_filtrado,
    on="SkIdInsumo", how="left", validate="m:1"
)

# === 5) CAPITULO PRESUPUESTO ===
capitulo_presupuesto = pd.read_csv(ruta_capitulos, low_memory=False)
columnas_capitulo = ["Codigo Proyecto", "Capitulo Descripcion"]
missing = [c for c in columnas_capitulo if c not in capitulo_presupuesto.columns]
if missing:
    raise KeyError(f"Faltan columnas en CapituloPresupuesto: {missing}")

capitulo_filtrado = capitulo_presupuesto[columnas_capitulo].rename(columns={
    "Codigo Proyecto": "Codigo_Proyecto",
    "Capitulo Descripcion": "Capitulo_Descripcion"
})

if "Codigo_Proyecto" not in tabla_intermedia.columns:
    proyecto_cod = proyecto_filtrado[["SkIdProyecto", "Codigo_Proyecto"]].drop_duplicates()
    tabla_intermedia = pd.merge(
        tabla_intermedia, proyecto_cod,
        on="SkIdProyecto", how="left", validate="m:1"
    )

tabla_final = pd.merge(
    tabla_intermedia, capitulo_filtrado,
    on="Codigo_Proyecto", how="left", validate="m:m"
)

# === 6) RELLENAR VALORES FALTANTES ===
tabla_final["MacroProyecto_Descripcion_Proyecto"] = tabla_final["MacroProyecto_Descripcion_Proyecto"].fillna("No pertenece a un Macro Proyecto")
tabla_final["Insumo_Descripcion"] = tabla_final["Insumo_Descripcion"].fillna("Descripcion No Disponible")
tabla_final["Fecha_Modificacion_Insumo"] = tabla_final["Fecha_Modificacion_Insumo"].fillna("2009-12-31")
tabla_final["Fecha_Creacion_Insumo"] = tabla_final["Fecha_Creacion_Insumo"].fillna("2009-12-31")

# === 7) CONVERTIR FECHAS A FORMATO ISO ===
def convertir_fecha(columna):
    """
    Convierte diferentes formatos de fecha al estándar ISO (YYYY-MM-DD)
    """
    fechas = pd.to_datetime(columna, errors="coerce", dayfirst=True)
    return fechas.dt.strftime("%Y-%m-%d")

tabla_final["Fecha_Creacion_Insumo"] = convertir_fecha(tabla_final["Fecha_Creacion_Insumo"])
tabla_final["Fecha_Modificacion_Insumo"] = convertir_fecha(tabla_final["Fecha_Modificacion_Insumo"])

# Reportar errores
for col in ["Fecha_Creacion_Insumo", "Fecha_Modificacion_Insumo"]:
    errores = tabla_final[col].isna().sum()
    print(f"🧮 {col}: {errores} valores no convertidos (NaT)")

print("\n✅ Fechas unificadas en formato YYYY-MM-DD.")


KeyboardInterrupt: 

In [ ]:

# === 8) GUARDAR RESULTADO FINAL ===
tabla_final.to_csv(ruta_salida, index=False)
print("\n✅ Proceso completado. Archivo guardado en:", ruta_salida)